In [1]:
import os
import glob
import pandas as pd
import pyarrow.parquet as pq
import geopandas as gp

from shapely.geometry import Point
from adjustText import adjust_text

import matplotlib
import matplotlib.pyplot as plt

In [2]:
os.getcwd()

'/home/prao/GIT/DataKind-Broadband/Notebooks'

In [3]:
data_dir = '../Data/shapefiles'

In [4]:
df_list = []
bb_type = 'fixed'
suffix = 'zip'
#suffix = 'parquet'
for year in [2021]:
    for file in glob.glob(data_dir + '/' + str(year) + '*_' + bb_type + '_*.' + suffix):
        print(file)
        #df = pq.read_table(source=file).to_pandas()
        df = gp.read_file(file)
        df['year'] = year
        df['bb_type'] = bb_type
        df_list.append(df)

In [6]:
df_fixed = pd.concat(df_list)

In [8]:
df_fixed.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 13336984 entries, 0 to 6694233
Data columns (total 9 columns):
 #   Column      Dtype   
---  ------      -----   
 0   quadkey     object  
 1   avg_d_kbps  int64   
 2   avg_u_kbps  int64   
 3   avg_lat_ms  int64   
 4   tests       int64   
 5   devices     int64   
 6   geometry    geometry
 7   year        int64   
 8   bb_type     object  
dtypes: geometry(1), int64(6), object(2)
memory usage: 1017.5+ MB


In [7]:
df_fixed.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,year,bb_type
0,3112231211112221,30246,14149,10,49,16,"POLYGON ((144.76135 -37.77940, 144.76685 -37.7...",2021,fixed
1,1321222320312202,232361,225471,4,1029,260,"POLYGON ((114.16992 22.36532, 114.17542 22.365...",2021,fixed
2,0313113232023121,95299,15138,12,96,20,"POLYGON ((-2.03796 52.60972, -2.03247 52.60972...",2021,fixed
3,1202212030113301,204895,69600,7,40,15,"POLYGON ((6.66321 46.51352, 6.66870 46.51352, ...",2021,fixed
4,1233013312311330,53680,51232,8,758,265,"POLYGON ((78.38745 17.44499, 78.39294 17.44499...",2021,fixed


In [19]:
# Load raw data

#df = pd.read_csv(data_file)


In [53]:
df_fixed[df_fixed['quadkey'] == '0231113112003202'].head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,year,bb_type
1495299,0231113112003202,221349,14691,11,17,6,"POLYGON ((-90.65918 38.49229, -90.65369 38.492...",2021,fixed
394990,0231113112003202,216548,15503,13,17,9,"POLYGON ((-90.65918 38.49229, -90.65369 38.492...",2021,fixed


In [54]:
df_fixed.year.unique()

array([2021])

In [55]:
df_fixed_yearly = df_fixed.groupby(['year','bb_type','quadkey','tile']).agg(
    avg_d_kbps = pd.NamedAgg(column='avg_d_kbps', aggfunc='mean'),
    avg_u_kbps = pd.NamedAgg(column='avg_u_kbps', aggfunc='mean'),
    avg_lat_ms = pd.NamedAgg(column='avg_lat_ms', aggfunc='mean'),
    tests = pd.NamedAgg(column='tests', aggfunc='mean'),
    devices = pd.NamedAgg(column='devices', aggfunc='mean')
).reset_index()

KeyError: 'tile'

In [39]:
df_fixed_yearly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29804382 entries, 0 to 29804381
Data columns (total 10 columns):
 #   Column      Dtype   
---  ------      -----   
 0   year        int64   
 1   bb_type     object  
 2   quadkey     object  
 3   tile        object  
 4   avg_d_kbps  float64 
 5   avg_u_kbps  float64 
 6   avg_lat_ms  float64 
 7   tests       float64 
 8   devices     float64 
 9   geometry    geometry
dtypes: float64(5), geometry(1), int64(1), object(3)
memory usage: 2.2+ GB


In [24]:
df_fixed_yearly.head()

,year,bb_type,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices
0,2019,fixed,0022133222312322,"POLYGON((-160.02685546875 70.6435894914449, -1...",8200.5,3705.0,53.0,1.0,1.00
1,2019,fixed,0022133222330013,"POLYGON((-160.032348632812 70.6399478155463, -...",9255.5,3189.5,43.5,1.0,1.00
2,2019,fixed,0022133222330023,"POLYGON((-160.043334960938 70.6363054807905, -...",7946.0,3492.5,44.5,1.5,1.25
3,2019,fixed,0022133222330032,"POLYGON((-160.037841796875 70.6363054807905, -...",7384.0,3868.0,65.0,1.0,1.00
4,2019,fixed,0022133222330100,"POLYGON((-160.02685546875 70.6417687358462, -1...",8895.0,3429.0,43.0,2.0,2.00


In [38]:
df_fixed_yearly['geometry'] = gp.GeoSeries.from_wkt(df['tile'])

In [25]:
# Florida State Code = 12

In [9]:
# zipfile of U.S. county boundaries
county_url = "https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip"
counties = gp.read_file(county_url)

In [10]:
# filter out the FL fips code
fl_counties = counties.loc[counties['STATEFP'] == '12'].to_crs(4326)

In [57]:
fl_counties.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
71,12,053,00295751,12053,Hernando,Hernando County,06,H1,G4020,None,45300,None,A,1224962206,300591492,+28.5730426,-082.4662272,"POLYGON ((-82.25329 28.47849, -82.25329 28.478..."
72,12,129,00306912,12129,Wakulla,Wakulla County,06,H1,G4020,None,45220,None,A,1570615733,334902363,+30.1394320,-084.3748463,"POLYGON ((-84.50434 30.30186, -84.50432 30.301..."
73,12,131,00295727,12131,Walton,Walton County,06,H1,G4020,None,18880,None,A,2687740188,522795160,+30.6312106,-086.1766139,"POLYGON ((-86.39160 30.72189, -86.39160 30.723..."
78,12,127,00306921,12127,Volusia,Volusia County,06,H1,G4020,422,19660,None,A,2852204344,857791723,+29.0577690,-081.1617920,"POLYGON ((-81.50280 29.10627, -81.50284 29.106..."
195,12,051,00307626,12051,Hendry,Hendry County,06,H1,G4020,163,17500,None,A,2994619266,87037213,+26.5399670,-081.1521142,"POLYGON ((-81.27172 26.42265, -81.27172 26.436..."


In [43]:
df_fl = pd.DataFrame(fl_counties)

In [11]:
tiles_in_fl = gp.sjoin(df_fixed, fl_counties, how='inner', op='intersects')

In [14]:
df_fl = pd.DataFrame(tiles_in_fl)

In [12]:
tiles_in_fl.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,year,bb_type,index_right,...,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
6,0320201332021332,90726,28194,35,3,2,"POLYGON ((-85.00122 29.73099, -84.99573 29.730...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742
57496,0320201332301131,64730,7275,43,8,8,"POLYGON ((-84.81995 29.67851, -84.81445 29.678...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742
100619,0320201332032000,183112,25706,30,7,5,"POLYGON ((-84.99023 29.72622, -84.98474 29.726...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742
129401,0320201332213220,70138,14725,28,8,3,"POLYGON ((-84.94629 29.62122, -84.94080 29.621...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742
130333,0320201332121101,17722,4729,28,7,1,"POLYGON ((-84.83093 29.76438, -84.82544 29.764...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742


In [13]:
tiles_in_fl.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 165093 entries, 6 to 2549416
Data columns (total 27 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   quadkey      165093 non-null  object  
 1   avg_d_kbps   165093 non-null  int64   
 2   avg_u_kbps   165093 non-null  int64   
 3   avg_lat_ms   165093 non-null  int64   
 4   tests        165093 non-null  int64   
 5   devices      165093 non-null  int64   
 6   geometry     165093 non-null  geometry
 7   year         165093 non-null  int64   
 8   bb_type      165093 non-null  object  
 9   index_right  165093 non-null  int64   
 10  STATEFP      165093 non-null  object  
 11  COUNTYFP     165093 non-null  object  
 12  COUNTYNS     165093 non-null  object  
 13  GEOID        165093 non-null  object  
 14  NAME         165093 non-null  object  
 15  NAMELSAD     165093 non-null  object  
 16  LSAD         165093 non-null  object  
 17  CLASSFP      165093 non-null  object  


In [15]:
df_fl.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,year,bb_type,index_right,...,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
6,0320201332021332,90726,28194,35,3,2,"POLYGON ((-85.00122 29.73099, -84.99573 29.730...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742
57496,0320201332301131,64730,7275,43,8,8,"POLYGON ((-84.81995 29.67851, -84.81445 29.678...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742
100619,0320201332032000,183112,25706,30,7,5,"POLYGON ((-84.99023 29.72622, -84.98474 29.726...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742
129401,0320201332213220,70138,14725,28,8,3,"POLYGON ((-84.94629 29.62122, -84.94080 29.621...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742
130333,0320201332121101,17722,4729,28,7,1,"POLYGON ((-84.83093 29.76438, -84.82544 29.764...",2021,fixed,1000,...,H1,G4020,None,None,None,A,1411908392,1275539662,+29.8101756,-084.7991742


In [44]:
df_fl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67 entries, 71 to 3206
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   67 non-null     object  
 1   COUNTYFP  67 non-null     object  
 2   COUNTYNS  67 non-null     object  
 3   GEOID     67 non-null     object  
 4   NAME      67 non-null     object  
 5   NAMELSAD  67 non-null     object  
 6   LSAD      67 non-null     object  
 7   CLASSFP   67 non-null     object  
 8   MTFCC     67 non-null     object  
 9   CSAFP     35 non-null     object  
 10  CBSAFP    51 non-null     object  
 11  METDIVFP  3 non-null      object  
 12  FUNCSTAT  67 non-null     object  
 13  ALAND     67 non-null     int64   
 14  AWATER    67 non-null     int64   
 15  INTPTLAT  67 non-null     object  
 16  INTPTLON  67 non-null     object  
 17  geometry  67 non-null     geometry
dtypes: geometry(1), int64(2), object(15)
memory usage: 9.9+ KB


In [17]:
df_fixed_yearly = df_fl.groupby(['year','bb_type','quadkey']).agg(
    avg_d_kbps = pd.NamedAgg(column='avg_d_kbps', aggfunc='median'),
    avg_u_kbps = pd.NamedAgg(column='avg_u_kbps', aggfunc='median'),
    avg_lat_ms = pd.NamedAgg(column='avg_lat_ms', aggfunc='median'),
    tests = pd.NamedAgg(column='tests', aggfunc='median'),
    devices = pd.NamedAgg(column='devices', aggfunc='median')
).reset_index()

In [33]:
#df_fl['geometry'] = df_fl['geometry'].astype(str)

In [45]:
#df_fl_fixed_yearly = pd.merge(df_fixed_yearly, df_fl, left_on='geometry', right_on='geometry', how='inner')

In [2]:
df_fixed_yearly.head()

NameError: name 'df_fixed_yearly' is not defined